# Dissonant Counterpoint and Statistical Feedback in James Tenney's Seegersongs

James Tenney's Seegersong #1 (solo clarinet) and Seegersong #2 (solo flute) explore the use of dissonant counterpoint in melodic writing, modelling the technique of avoiding pitch repetition used by Ruth Crawford Seeger. Tenney developed an algorithm for tracking and calculating the probability of pitch recurrence. The Dissonant Counterpoint Algorithm (DCA) is the name attributed by Larry Polansky, Alex Barnett, and Michael Winter (A Few More Words About James Tenney:
Dissonant Counterpoint and Statistical Feedback, http://eamusic.dartmouth.edu/~larry/published_articles/dc_nov_6.pdf). This algorithm appeared in different forms throughout much of Tenney's later work, though he showed a musicological interest at least as early as the 70's in his paper on Carl Ruggles (The Chronological Development of Carl Ruggles' Melodic Style, Perspectives of New Music, Vol. 16, No. 1 (Autumn - Winter, 1977), pp. 36-69). 

Just like in serialism, this technique can be applied to any computable musical parameter, but in Seegersongs it is used clearly to determine pitch material in a chromatic field.

My purpose here is to present simple examples of the use of statistical feedback in randomized pitch selection similar to Tenney's methods in Seegersongs and the way parameters may be manipulated for different types of output. Scrolling down through the text and code, various widgets will appear throughout that will allow you to change individual parameters and generate output which will print beneath each cell. If you edit any of the code, you will need to rerun that cell by pressing shift-enter with the cell selected or pressing "Run" in the top menu.

In [7]:
# imports

import ipywidgets as widgets
from IPython.display import clear_output
import dca_functions

These first examples will generate use numeric output which could be applied to any parameter but here will be assumed to be in regard to pitches for simplicity (in this case chromatic pitches, but could be partials, frequencies, etc.). As stated above, this data could also be used to map various other musical parameters (timing, dynamics, voicing, etc.).

In [8]:
# default declaration, can change with slider/button
pitches = list(range(0, 12))

pitch_range_slider = widgets.IntRangeSlider(
    value=[0, 11],
    min=0,
    max=11, # make this larger to include more than one octave
    description='Pitch Range:', # chromatic pitches
)

print("Specify pitches to build pitch sequence from (0 is Middle C)")
display(pitch_range_slider)

# can change this to offset from middle C
offset = 0

# Number of pitches in sequence
length = widgets.IntSlider(
    value=50,
    min=1,
    max=500,
    step=1,
    description='Length (# notes):',
    style={'description_width': 'initial', 'width': '800px'},
    readout=True,
)
print("Specify length of sequence in notes:")
display(length)
    


Specify pitches to build pitch sequence from (0 is Middle C)


IntRangeSlider(value=(0, 11), description='Pitch Range:', max=11)

Specify length of sequence in notes:


IntSlider(value=50, description='Length (# notes):', max=500, min=1, style=SliderStyle(description_width='init…

Although each pitch is generated randomly, the DCA differs from an independent and identically distributed random process (Polansky, Barnett, Winter, p. 5) in that it guarantees a more uniform local and global random distribution. In a random process like rolling a die, it is not uncommon to roll the same number multiple times in a row, so looking at just a few rolls might not give an accurate picture of the process. Clicking the button below after running the next cell will generate random pitch sequences, though there will likely be some repetitions. Try clicking a few times and compare individual parts of the sequence to see how the distribution varies dramatically at times.

In [9]:
random_PS_button = widgets.Button(
    description='Make random pitch sequence',
    layout=widgets.Layout(width='200px'),
)
output = widgets.Output()

display(random_PS_button, output)

def on_button_clicked(b):
    with output:
        clear_output()
        print(dca_functions.make_random_pitch_sequence(offset, length, pitch_range_slider))

random_PS_button.on_click(on_button_clicked)

Button(description='Make random pitch sequence', layout=Layout(width='200px'), style=ButtonStyle())

Output()

The DCA keeps track of how long it has been since each element (pitch in this case) has been selected and calculates the probability of it being selected next, based on that count. A growth function can be specified to control how this count affects the probability (Tenney used an exponential function to increase the probability of high counts being selected). A weight parameter can be specified to affect the probability of individual elements (Tenney determined a pitch center which moved over time and calculated the weight of the other pitch elements by their proximity to this center). 

Try adjusting the above parameters and clicking the button to generate different sequences. The weight option "equal" will equally favor each element. "favor lower" and "favor upper" will favor one side or the other of the pitch range, while "favor center" will diminish the weight going up and down from the center (so the highest and lowest elements will have the lowest weight).

Read through some of the data output. Each random note selection is printed with the counts for each element and their current probabilities. When a pitch element is selected, its count is set to zero and will increase with each future selection until it's selected again. Its probability will be the lowest right after being selected. Scroll to the bottom and look at the full sequence. Are there any direct repetitions? Probably not. There is still a small chance for direct repetitions to occur, but they will be very unlikely. Look at the pitch frequency for each element from the sequence. An equal weight should mean that the pitch frequencies are nearly identical. A favored weighting should be reflected as well. Try adjusting the sequence length. While very short sequences might still have some weird variations, the distribution should be much closer to that of the long sequences than in the iid random sequences.

In [10]:
dca_PS_button1 = widgets.Button(
    description='Make dissonant counterpoint spaced pitch sequence',
    layout=widgets.Layout(width='350px'),
)

dca_PS_button2 = widgets.Button(
    description='Make dissonant counterpoint spaced pitch sequence',
    layout=widgets.Layout(width='350px'),
)

display(pitch_range_slider)
display(length)

weights_option = widgets.Dropdown(
    options=['equal', 'favor lower', 'favor center', 'favor upper'],
    value='equal',
    description='Weight options:',
    style={'description_width': 'initial', 'width': '800px'},
)

print_option = widgets.ToggleButtons(
    options=["Don't Print", "Print Notation"],
    description='Print Option:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

display(weights_option)

display(dca_PS_button1, output)

def on_dca_button_clicked(b):
    with output:
        clear_output()
        if print_option.value == "Print Notation":
            print_ly = True
            show_data = False
        else:
            print_ly = False
            show_data = True
        sequence = dca_functions.make_dca_spaced_pitch_sequence(offset, weights_option, length, pitch_range_slider, show_data=show_data, print_ly=print_ly)
        print("Sequence:", sequence)
        
        pitches = list(range(pitch_range_slider.value[0], pitch_range_slider.value[1]+1))
        print("\nPitch Frequency (total times each pitch occurs in sequence:")
        for pitch in pitches:
            print(pitch, ":", sequence.count(pitch))
        
dca_PS_button1.on_click(on_dca_button_clicked)
dca_PS_button2.on_click(on_dca_button_clicked)

IntRangeSlider(value=(0, 11), description='Pitch Range:', max=11)

IntSlider(value=50, description='Length (# notes):', max=500, min=1, style=SliderStyle(description_width='init…

Dropdown(description='Weight options:', options=('equal', 'favor lower', 'favor center', 'favor upper'), style…

Button(description='Make dissonant counterpoint spaced pitch sequence', layout=Layout(width='350px'), style=Bu…

Output()

Now let's see what this looks like in notation. Set the print option to "Print Notation" and create a new sequence. After a moment a score image will display, followed by the numeric output and frequency counts.

In [11]:
display(print_option)
display(pitch_range_slider)
display(length)
display(weights_option)
display(dca_PS_button2, output)

ToggleButtons(description='Print Option:', options=("Don't Print", 'Print Notation'), value="Don't Print")

IntRangeSlider(value=(0, 11), description='Pitch Range:', max=11)

IntSlider(value=50, description='Length (# notes):', max=500, min=1, style=SliderStyle(description_width='init…

Dropdown(description='Weight options:', options=('equal', 'favor lower', 'favor center', 'favor upper'), style…

Button(description='Make dissonant counterpoint spaced pitch sequence', layout=Layout(width='350px'), style=Bu…

Output()

# How about changing over time?

Following the general form of Seegersong #1, we can create a similar piece which:

1. makes weights a function of their distance from a pitch center
2. moves that pitch center over time
3. makes sequences vary in length (ending each one with a rest)
4. increases pitch center and sequence average length until 2/3 through form, then decreases

Click the button below to generate a piece. It will take a minute to generate the notation. Scroll down to view everything.

In [12]:
dca_piece_button = widgets.Button(
    description='Make dissonant counterpoint piece',
    layout=widgets.Layout(width='350px'),
)

def on_dca_button3_clicked(b):
    with output:
        clear_output()      
        # build clangs
        total_clangs = 72

        # weight by proximity to pitch center
        weights_option.value = "favor center"

        # make peak 2/3 of the way through form
        peak = int(total_clangs * (2/3))

        initial_length = 5
        undeviated_length = initial_length
        length.value = initial_length # length of initial sequence
        length_deviation = int(length.value * 0.5)

        offset = -12
        pitch_center = 6
        clangs = []

        for clang in range(total_clangs):
            pitches = [*range(int(pitch_center) - 6, int(pitch_center+6))]
            #print(pitches)

            sequence = dca_functions.make_clang(offset, length_deviation, weights_option, undeviated_length, length, pitch_range_slider, pitches=pitches)
            clangs.append(sequence)

            if clang < peak:
                pitch_center += 0.6
                undeviated_length += 1
            else:
                pitch_center -= 1.25
                undeviated_length -= 2
                length_deviation = int(length.value * 0.5)

        for clang in clangs:
            print(clang)

        dca_functions.output_clangs_ly(clangs)
        
display(dca_piece_button, output)

dca_piece_button.on_click(on_dca_button3_clicked)

Button(description='Make dissonant counterpoint piece', layout=Layout(width='350px'), style=ButtonStyle())

Output()